In [1]:
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests
import ast
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php'

In [3]:
soup = BeautifulSoup(requests.get(url).content)

In [4]:
for mun in soup.find_all('div',{'class' : 'container-codigos'}):
    print(mun.find('td',{'class': 'numero'}))

<td class="numero"><a href="#AC">12<span class="saiba-mais-tag">ver municípios</span></a></td>


In [5]:
boody_num = soup.find('div',{'class' : 'container-codigos'})

In [6]:
#  index 27 is when begin munip codes 
boody_num.find_all('td',{'class': 'numero'})[27:28]

[<td class="numero">1200013</td>]

In [7]:
munip = []
for mun in boody_num.find_all('td',{'class': 'numero'})[27:]:
    munip.append(mun.text)

In [8]:
len(munip)

5570

In [9]:
url_api = 'http://www.transparencia.gov.br/api-de-dados/bolsa-familia-por-municipio?mesAno=201902&codigoIbge=3550308&pagina=1'
# Format view
requests.get(url_api).json()

[{'id': 69002750,
  'dataReferencia': '01/02/2019',
  'municipio': {'codigoIBGE': '3550308',
   'nomeIBGE': 'SÃO PAULO',
   'nomeIBGEsemAcento': 'SAO PAULO',
   'pais': 'BRASIL',
   'uf': {'sigla': 'SP', 'nome': 'SÃO PAULO'}},
  'tipo': {'id': 1,
   'descricao': 'Bolsa Família',
   'descricaoDetalhada': 'Bolsa Família'},
  'valor': 73800539.0,
  'quantidadeBeneficiados': 450885}]

In [10]:
lst = []

In [11]:
def gattering(munip,lst,i=0):
    '''
    This function receives a list of municipalities and a empty list,
    making request from a API and in the same time creating a DataFrame to make a bkp into a csv file.
    When getting a error, he will repeat the method after 61 seconds.
    '''
    for pos, mun in enumerate(munip[i:],i):
        print("Municipio: ", mun , 'Lista[' , pos , ']')
        try:
            url_api = f'http://www.transparencia.gov.br/api-de-dados/bolsa-familia-por-municipio?mesAno=201911&codigoIbge={mun}&pagina=1'
            lst.append(requests.get(url_api).json())
            data = pd.DataFrame(lst)
            data.to_csv('dados.csv')
            time.sleep(1)
        except:
            time.sleep(61)
            gattering(munip,lst,pos)
    return lst

In [12]:
# Gathering the data
# lst_df = gattering(munip,lst,0)

In [13]:
def flatten(df, col_names):
    
    for col_name in col_names:
        flattened = pd.DataFrame.from_records(df[col_name])
        flattened.columns = [f"{col_name}_{col}" for col in flattened.columns]
        df = pd.concat([df, flattened], axis=1)
        df = df.drop(col_name, axis=1)
        
    return df

# Bakcup - if needed

In [14]:
back = pd.read_csv('dados.csv')

In [15]:
back.drop('Unnamed: 0',axis=1,inplace=True)
back.head()

,0
0,"{'id': 75202990, 'dataReferencia': '01/11/2019..."
1,"{'id': 69202460, 'dataReferencia': '01/11/2019..."
2,"{'id': 71003156, 'dataReferencia': '01/11/2019..."
3,"{'id': 75803233, 'dataReferencia': '01/11/2019..."
4,"{'id': 67505039, 'dataReferencia': '01/11/2019..."


In [19]:
# Recovering the list format from backup csv
# ast.literal_eval() - recover a string to dictionary - a string with double quotes " "
lst = [item for item in back['0']]
lst_recover = []
for pos, item in enumerate(lst,0):
    if item != 'Error API':
        # here is where I return my data to a json format from a string
        lst_recover.append(ast.literal_eval(item))    

In [21]:
lst_recover[0]

{'id': 75202990,
 'dataReferencia': '01/11/2019',
 'municipio': {'codigoIBGE': '1200013',
  'nomeIBGE': 'ACRELÂNDIA',
  'nomeIBGEsemAcento': 'ACRELANDIA',
  'pais': 'BRASIL',
  'uf': {'sigla': 'AC', 'nome': 'ACRE'}},
 'tipo': {'id': 1,
  'descricao': 'Bolsa Família',
  'descricaoDetalhada': 'Bolsa Família'},
 'valor': 307671.0,
 'quantidadeBeneficiados': 1563}

In [22]:
#  Making every json to a list member so I can append later in a dataframe
lst_final = [[item] for item in lst_recover]
lst_final[0]

[{'id': 75202990,
  'dataReferencia': '01/11/2019',
  'municipio': {'codigoIBGE': '1200013',
   'nomeIBGE': 'ACRELÂNDIA',
   'nomeIBGEsemAcento': 'ACRELANDIA',
   'pais': 'BRASIL',
   'uf': {'sigla': 'AC', 'nome': 'ACRE'}},
  'tipo': {'id': 1,
   'descricao': 'Bolsa Família',
   'descricaoDetalhada': 'Bolsa Família'},
  'valor': 307671.0,
  'quantidadeBeneficiados': 1563}]

In [23]:
# Missing 5 munip
errors_list = []
for pos, item in enumerate(lst,0):
    if item == 'Error API':
        print(pos)
        errors_list.append(pos)

989
1674
2819
3114
3144


In [24]:
munip[989]

'5209937'

In [25]:
# lst_recover 5 missing munip
len(lst_recover)

5565

In [26]:
lst[989]

'Error API'

In [29]:
# Retriving lost data. With a small number of lost data I can retrieve manually these informations
for pos, item in enumerate(lst, 0):
    if item == 'Error API':
        url_api = f'http://www.transparencia.gov.br/api-de-dados/bolsa-familia-por-municipio?mesAno=201911&codigoIbge={munip[pos]}&pagina=1'
        lst_final.append(requests.get(url_api).json())

In [30]:
# Now I have all munip
len(lst_final)

5570

# Making the Dataframe

In [31]:
# Now with all my json files I can create my dataframe
data = pd.DataFrame(lst_final,columns=['data'])
data.head()

,data
0,"{'id': 75202990, 'dataReferencia': '01/11/2019..."
1,"{'id': 69202460, 'dataReferencia': '01/11/2019..."
2,"{'id': 71003156, 'dataReferencia': '01/11/2019..."
3,"{'id': 75803233, 'dataReferencia': '01/11/2019..."
4,"{'id': 67505039, 'dataReferencia': '01/11/2019..."


In [32]:
# flattening the data
data = flatten(data, ['data'])

In [33]:
data.head()

,data_id,data_dataReferencia,data_municipio,data_tipo,data_valor,data_quantidadeBeneficiados
0,75202990,01/11/2019,"{'codigoIBGE': '1200013', 'nomeIBGE': 'ACRELÂN...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",307671.0,1563
1,69202460,01/11/2019,"{'codigoIBGE': '1200054', 'nomeIBGE': 'ASSIS B...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",443147.0,1336
2,71003156,01/11/2019,"{'codigoIBGE': '1200104', 'nomeIBGE': 'BRASILÉ...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",615761.0,3030
3,75803233,01/11/2019,"{'codigoIBGE': '1200138', 'nomeIBGE': 'BUJARI'...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",305647.0,1326
4,67505039,01/11/2019,"{'codigoIBGE': '1200179', 'nomeIBGE': 'CAPIXAB...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",367782.0,1444


In [34]:
data = flatten(data, ["data_municipio", "data_tipo"])
data.head()

,data_id,data_dataReferencia,data_valor,data_quantidadeBeneficiados,data_municipio_codigoIBGE,data_municipio_nomeIBGE,data_municipio_nomeIBGEsemAcento,data_municipio_pais,data_municipio_uf,data_tipo_id,data_tipo_descricao,data_tipo_descricaoDetalhada
0,75202990,01/11/2019,307671.0,1563,1200013,ACRELÂNDIA,ACRELANDIA,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
1,69202460,01/11/2019,443147.0,1336,1200054,ASSIS BRASIL,ASSIS BRASIL,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
2,71003156,01/11/2019,615761.0,3030,1200104,BRASILÉIA,BRASILEIA,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
3,75803233,01/11/2019,305647.0,1326,1200138,BUJARI,BUJARI,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
4,67505039,01/11/2019,367782.0,1444,1200179,CAPIXABA,CAPIXABA,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família


In [35]:
data = flatten(data,['data_municipio_uf'])
data.head()

,data_id,data_dataReferencia,data_valor,data_quantidadeBeneficiados,data_municipio_codigoIBGE,data_municipio_nomeIBGE,data_municipio_nomeIBGEsemAcento,data_municipio_pais,data_tipo_id,data_tipo_descricao,data_tipo_descricaoDetalhada,data_municipio_uf_sigla,data_municipio_uf_nome
0,75202990,01/11/2019,307671.0,1563,1200013,ACRELÂNDIA,ACRELANDIA,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
1,69202460,01/11/2019,443147.0,1336,1200054,ASSIS BRASIL,ASSIS BRASIL,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
2,71003156,01/11/2019,615761.0,3030,1200104,BRASILÉIA,BRASILEIA,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
3,75803233,01/11/2019,305647.0,1326,1200138,BUJARI,BUJARI,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
4,67505039,01/11/2019,367782.0,1444,1200179,CAPIXABA,CAPIXABA,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE


Final Format

In [36]:
df = data[['data_dataReferencia','data_municipio_nomeIBGE','data_municipio_uf_sigla','data_tipo_descricao','data_quantidadeBeneficiados','data_valor']].copy()
df.head()

,data_dataReferencia,data_municipio_nomeIBGE,data_municipio_uf_sigla,data_tipo_descricao,data_quantidadeBeneficiados,data_valor
0,01/11/2019,ACRELÂNDIA,AC,Bolsa Família,1563,307671.0
1,01/11/2019,ASSIS BRASIL,AC,Bolsa Família,1336,443147.0
2,01/11/2019,BRASILÉIA,AC,Bolsa Família,3030,615761.0
3,01/11/2019,BUJARI,AC,Bolsa Família,1326,305647.0
4,01/11/2019,CAPIXABA,AC,Bolsa Família,1444,367782.0


In [37]:
df.shape

(5570, 6)

# Another way to create the final dataset

In [38]:
df = pd.DataFrame(lst_final[0])
df

,id,dataReferencia,municipio,tipo,valor,quantidadeBeneficiados
0,75202990,01/11/2019,"{'codigoIBGE': '1200013', 'nomeIBGE': 'ACRELÂN...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",307671.0,1563


In [39]:
# Using lst_final[1:] because the dataframe is already created from the lst_final[0]
# Then now I only going to append the data
for data in lst_final[1:]:
    df= df.append(data)

In [40]:
df.head()

,id,dataReferencia,municipio,tipo,valor,quantidadeBeneficiados
0,75202990,01/11/2019,"{'codigoIBGE': '1200013', 'nomeIBGE': 'ACRELÂN...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",307671.0,1563
0,69202460,01/11/2019,"{'codigoIBGE': '1200054', 'nomeIBGE': 'ASSIS B...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",443147.0,1336
0,71003156,01/11/2019,"{'codigoIBGE': '1200104', 'nomeIBGE': 'BRASILÉ...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",615761.0,3030
0,75803233,01/11/2019,"{'codigoIBGE': '1200138', 'nomeIBGE': 'BUJARI'...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",305647.0,1326
0,67505039,01/11/2019,"{'codigoIBGE': '1200179', 'nomeIBGE': 'CAPIXAB...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",367782.0,1444


In [41]:
# Now remaking the index
df.reset_index(inplace=True)
df.drop(columns='index',inplace=True)

In [42]:
df.head()

,id,dataReferencia,municipio,tipo,valor,quantidadeBeneficiados
0,75202990,01/11/2019,"{'codigoIBGE': '1200013', 'nomeIBGE': 'ACRELÂN...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",307671.0,1563
1,69202460,01/11/2019,"{'codigoIBGE': '1200054', 'nomeIBGE': 'ASSIS B...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",443147.0,1336
2,71003156,01/11/2019,"{'codigoIBGE': '1200104', 'nomeIBGE': 'BRASILÉ...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",615761.0,3030
3,75803233,01/11/2019,"{'codigoIBGE': '1200138', 'nomeIBGE': 'BUJARI'...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",305647.0,1326
4,67505039,01/11/2019,"{'codigoIBGE': '1200179', 'nomeIBGE': 'CAPIXAB...","{'id': 1, 'descricao': 'Bolsa Família', 'descr...",367782.0,1444


In [43]:
df = flatten(df, ["municipio", "tipo"])
df.head()

,id,dataReferencia,valor,quantidadeBeneficiados,municipio_codigoIBGE,municipio_nomeIBGE,municipio_nomeIBGEsemAcento,municipio_pais,municipio_uf,tipo_id,tipo_descricao,tipo_descricaoDetalhada
0,75202990,01/11/2019,307671.0,1563,1200013,ACRELÂNDIA,ACRELANDIA,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
1,69202460,01/11/2019,443147.0,1336,1200054,ASSIS BRASIL,ASSIS BRASIL,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
2,71003156,01/11/2019,615761.0,3030,1200104,BRASILÉIA,BRASILEIA,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
3,75803233,01/11/2019,305647.0,1326,1200138,BUJARI,BUJARI,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família
4,67505039,01/11/2019,367782.0,1444,1200179,CAPIXABA,CAPIXABA,BRASIL,"{'sigla': 'AC', 'nome': 'ACRE'}",1,Bolsa Família,Bolsa Família


In [44]:
df = flatten(df, ["municipio_uf"])
df.head()

,id,dataReferencia,valor,quantidadeBeneficiados,municipio_codigoIBGE,municipio_nomeIBGE,municipio_nomeIBGEsemAcento,municipio_pais,tipo_id,tipo_descricao,tipo_descricaoDetalhada,municipio_uf_sigla,municipio_uf_nome
0,75202990,01/11/2019,307671.0,1563,1200013,ACRELÂNDIA,ACRELANDIA,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
1,69202460,01/11/2019,443147.0,1336,1200054,ASSIS BRASIL,ASSIS BRASIL,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
2,71003156,01/11/2019,615761.0,3030,1200104,BRASILÉIA,BRASILEIA,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
3,75803233,01/11/2019,305647.0,1326,1200138,BUJARI,BUJARI,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE
4,67505039,01/11/2019,367782.0,1444,1200179,CAPIXABA,CAPIXABA,BRASIL,1,Bolsa Família,Bolsa Família,AC,ACRE


In [45]:
df_final = df[['dataReferencia','municipio_nomeIBGE','municipio_uf_sigla','tipo_descricao','quantidadeBeneficiados','valor']].copy()
df_final.head()

,dataReferencia,municipio_nomeIBGE,municipio_uf_sigla,tipo_descricao,quantidadeBeneficiados,valor
0,01/11/2019,ACRELÂNDIA,AC,Bolsa Família,1563,307671.0
1,01/11/2019,ASSIS BRASIL,AC,Bolsa Família,1336,443147.0
2,01/11/2019,BRASILÉIA,AC,Bolsa Família,3030,615761.0
3,01/11/2019,BUJARI,AC,Bolsa Família,1326,305647.0
4,01/11/2019,CAPIXABA,AC,Bolsa Família,1444,367782.0


In [46]:
# df.to_csv('final_format.csv')

In [47]:
df.groupby(['municipio_uf_sigla'])[['quantidadeBeneficiados', 'valor']].sum().reset_index()

,municipio_uf_sigla,quantidadeBeneficiados,valor
0,AC,82754,23778672.0
1,AL,373976,76039758.0
2,AM,366715,90121807.0
3,AP,70139,17140023.0
4,BA,1689947,332058293.0
5,CE,982378,194196981.0
6,DF,61517,12110463.0
7,ES,156301,29878861.0
8,GO,262302,44768677.0
9,MA,897598,201248934.0


In [48]:
df[df['municipio_uf_sigla'] == 'SP'][['quantidadeBeneficiados']].sum()

quantidadeBeneficiados    1263725
dtype: int64

In [49]:
# Same as above, but this time using the .pivot_table function
state = df.pivot_table(values=['quantidadeBeneficiados', 'valor'],
              index='municipio_uf_sigla',
              aggfunc='sum').reset_index()

In [50]:
state

,municipio_uf_sigla,quantidadeBeneficiados,valor
0,AC,82754,23778672.0
1,AL,373976,76039758.0
2,AM,366715,90121807.0
3,AP,70139,17140023.0
4,BA,1689947,332058293.0
5,CE,982378,194196981.0
6,DF,61517,12110463.0
7,ES,156301,29878861.0
8,GO,262302,44768677.0
9,MA,897598,201248934.0


In [51]:
def regiao_brasil(lst):
    lst_norte = ['AC','AP','AM','PA','RO','RR','TO']
    lst_nordeste = ['AL','BA','CE','MA','PB','PI','PE','RN','SE']
    lst_centro = ['DF','GO','MT','MS']
    lst_sudeste = ['ES','MG','RJ','SP']
    lst_sul = ['PR','RS','SC']
#     for sigla in lst:
    if lst in lst_norte:
        return 'Norte'
    elif lst in lst_nordeste:
        return 'Nordeste'
    elif lst in lst_centro:
        return 'Centro-Oeste'
    elif lst in lst_sudeste:
        return 'Sudeste'
    elif lst in lst_sul:
        return 'Sul'

In [52]:
state['regiao'] = state['municipio_uf_sigla'].apply(regiao_brasil)

In [53]:
state.sort_values('regiao',inplace=True)

In [54]:
state.reset_index(drop='index',inplace=True)

In [55]:
state

,municipio_uf_sigla,quantidadeBeneficiados,valor,regiao
0,DF,61517,12110463.0,Centro-Oeste
1,GO,262302,44768677.0,Centro-Oeste
2,MS,107640,21036205.0,Centro-Oeste
3,MT,135579,24709549.0,Centro-Oeste
4,AL,373976,76039758.0,Nordeste
5,SE,270900,48279674.0,Nordeste
6,BA,1689947,332058293.0,Nordeste
7,CE,982378,194196981.0,Nordeste
8,MA,897598,201248934.0,Nordeste
9,RN,328599,64434084.0,Nordeste


In [56]:
state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
municipio_uf_sigla        27 non-null object
quantidadeBeneficiados    27 non-null int64
valor                     27 non-null float64
regiao                    27 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 992.0+ bytes


In [57]:
state['valor'] = state['valor'].astype('Int64')

In [58]:
state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
municipio_uf_sigla        27 non-null object
quantidadeBeneficiados    27 non-null int64
valor                     27 non-null Int64
regiao                    27 non-null object
dtypes: Int64(1), int64(1), object(2)
memory usage: 1019.0+ bytes


In [53]:
state.groupby('regiao').sum().reset_index()

,regiao,quantidadeBeneficiados,valor
0,Centro-Oeste,567038,102624894
1,Nordeste,6532522,1321841041
2,Norte,1611217,361105972
3,Sudeste,3080724,602629727
4,Sul,716633,132009325


In [54]:
region = state.pivot_table(values=['quantidadeBeneficiados', 'valor'],
                           index='regiao',
                           aggfunc='sum').reset_index()

In [55]:
region

,regiao,quantidadeBeneficiados,valor
0,Centro-Oeste,567038,102624894
1,Nordeste,6532522,1321841041
2,Norte,1611217,361105972
3,Sudeste,3080724,602629727
4,Sul,716633,132009325
